In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [3]:
df = pd.read_csv("cleaned_results.csv")

In [65]:
df.tail(20)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
4864,2022-07-16,Denmark,Spain,0,1,UEFA Euro,London,Denmark,True,2022
4865,2022-07-09,Portugal,Switzerland,2,2,UEFA Euro,Leigh,Portugal,True,2022
4866,2022-07-09,Netherlands,Sweden,1,1,UEFA Euro,Sheffield,Netherlands,True,2022
4867,2022-07-13,Sweden,Switzerland,2,1,UEFA Euro,Sheffield,Sweden,True,2022
4868,2022-07-13,Netherlands,Portugal,3,2,UEFA Euro,Leigh,Netherlands,True,2022
4869,2022-07-17,Switzerland,Netherlands,1,4,UEFA Euro,Sheffield,Switzerland,True,2022
4870,2022-07-17,Sweden,Portugal,5,0,UEFA Euro,Leigh,Sweden,True,2022
4871,2022-07-10,Belgium,Iceland,1,1,UEFA Euro,Manchester,Belgium,True,2022
4872,2022-07-10,France,Italy,5,1,UEFA Euro,Rotherham,France,True,2022
4873,2022-07-14,Italy,Iceland,1,1,UEFA Euro,Manchester,Italy,True,2022


In [4]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False,1969
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True,1969
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True,1969
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False,1969
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True,1975


In [5]:
features_df = df.drop(columns=["home_score", "away_score", "date"])
y_home = df["home_score"]
y_away = df["away_score"]

In [6]:
import category_encoders

In [7]:
### Frequency Encoding
features_df_copy = features_df.copy()
columns_list = features_df.columns

#encode all columns
for col in columns_list:
  #encoder
  encoder = category_encoders.CountEncoder(cols = col, normalize=True)

  ##fit and transform
  features_df[col] = encoder.fit_transform(features_df[col])

features_df.head()

,home_team,away_team,tournament,city,country,neutral,year
0,0.022318,0.021908,0.004095,0.000205,0.016994,0.556511,0.000819
1,0.024570,0.020680,0.004095,0.000205,0.016994,0.443489,0.000819
2,0.022113,0.021908,0.004095,0.000614,0.016994,0.443489,0.000819
3,0.022318,0.030917,0.004095,0.000614,0.016994,0.556511,0.000819
4,0.010442,0.013309,0.054873,0.011671,0.011671,0.443489,0.002048


## Hypertuning parameters for home score

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features_df, y_home, test_size=0.2, random_state=42)

In [42]:
model = RandomForestRegressor(random_state=42)

In [43]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

In [44]:
print("Mean-squared error:", mse)

Mean-squared error: 4.04388146928473


In [45]:
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

R-squared score: 0.44605494687083147


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [47]:
#Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]}
scoring = {
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score)}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, refit='MSE', cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_mse = grid_search.best_score_
best_r2 = grid_search.cv_results_['mean_test_R2'][grid_search.best_index_]
best_model = RandomForestRegressor(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Best parameters:", best_params)
print("Best MSE:", best_mse)
print("Best R2:", best_r2)
print("MSE on test set:", mse)
print("R2 on test set:", r2)

Best parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best MSE: 4.526806521611325
Best R2: 0.39976987276085557
MSE on test set: 4.076030657495665
R2 on test set: 0.4416510384212804


In [48]:
model = RandomForestRegressor(n_estimators=100,max_depth=5, min_samples_split=10, min_samples_leaf=1, random_state=42)

In [49]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean-squared error:", mse)
print("R-squared score:", r2)

Mean-squared error: 4.0753167965730155
R-squared score: 0.44174882559178175


In [20]:
X_train, X_test, y_train, y_test = train_test_split(features_df, y_away, test_size=0.2, random_state=42)

In [14]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean-squared error:", mse)
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

Mean-squared error: 2.8864719026174868
R-squared score: 0.24106995899253936


## Hypertuning parameters for away score

In [16]:
import time

In [17]:
#Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]}
scoring = {
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score)}
print(time.time())
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, refit='MSE', cv=5)
grid_search.fit(X_train, y_train)
print(time.time())
best_params = grid_search.best_params_
best_mse = grid_search.best_score_
best_r2 = grid_search.cv_results_['mean_test_R2'][grid_search.best_index_]
best_model = RandomForestRegressor(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Best parameters:", best_params)
print("Best MSE:", best_mse)
print("Best R2:", best_r2)
print("MSE on test set:", mse)
print("R2 on test set:", r2)

1687152488.398727
1687152994.367168
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best MSE: 2.8407984736759015
Best R2: 0.27435718364502354
MSE on test set: 2.864652179713339
R2 on test set: 0.24680694301909445


In [18]:
model = RandomForestRegressor(n_estimators=100,max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean-squared error:", mse)
print("R-squared score:", r2)

Mean-squared error: 2.8864719026174868
R-squared score: 0.24106995899253936


## Predicting the score

In [48]:
def prediction(X):
    hX_train, hX_test, hy_train, hy_test = train_test_split(features_df, y_home, test_size=0.2, random_state=42)
    aX_train, aX_test, ay_train, ay_test = train_test_split(features_df, y_away, test_size=0.2, random_state=42)
    hmodel = RandomForestRegressor(n_estimators=100,max_depth=5, min_samples_split=10, min_samples_leaf=1, random_state=42)
    amodel = RandomForestRegressor(n_estimators=100,max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)
    hmodel.fit(hX_train, hy_train)
    amodel.fit(aX_train, ay_train)
    hy_pred = hmodel.predict(X)
    ay_pred = amodel.predict(X)
    print("Score:", np.round(hy_pred).astype(int)
, ":",np.round(ay_pred).astype(int)
 )
    

In [85]:
X=features_df.iloc[111]
X=pd.DataFrame(X)
a=X.values
values= a.flatten().tolist()
values

[0.0038902538902538903,
 0.005733005733005733,
 0.054873054873054876,
 0.009009009009009009,
 0.011466011466011465,
 0.44348894348894347,
 0.011056511056511056]

In [86]:
input_row = pd.DataFrame([values], columns=['home_team', 'away_team', 'tournament', 'city', 'country','neutral','year'])


In [74]:
prediction(input_row)

Score: [1] : [1]


In [77]:
prediction(input_row)

Score: [6] : [0]


In [82]:
prediction(input_row)

Score: [5] : [0]


In [87]:
prediction(input_row)

Score: [2] : [0]
